In [1]:
pip install pandas numpy scikit-learn joblib


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image

train_dir = r"C:\Users\VINIL\Desktop\soil_type_ML\notebooks\Soil Classification.v4i.folder\train"
test_dir = r"C:\Users\VINIL\Desktop\soil_type_ML\notebooks\Soil Classification.v4i.folder\test"

# 🔹 Data Preprocessing & Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation data from train
)

# 🔹 Load Training & Validation Data
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset="training"
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset="validation"
)

# 🔹 Load Test Data (No augmentation, just normalization)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# 🔹 Build the CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes
])

# 🔹 Compile Model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 🔹 Train the Model
epochs = 10  # Increase for better results
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

# 🔹 Evaluate Model on Test Dataa
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# 🔹 Save the Model
# model.save("soil_classification_model.h5")
print("Model saved successfully!")a

# 🔹 Function to Predict Single Image
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    print(f"Predicted Class: {predicted_class}")



Found 1880 images belonging to 5 classes.
Found 467 images belonging to 5 classes.
Found 80 images belonging to 5 classes.


c:\Users\VINIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\VINIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - accuracy: 0.5465 - loss: 1.5511 - val_accuracy: 0.8073 - val_loss: 0.5062
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.8380 - loss: 0.4823 - val_accuracy: 0.7880 - val_loss: 0.5632
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 52s 849ms/step - accuracy: 0.8744 - loss: 0.3636 - val_accuracy: 0.7923 - val_loss: 0.8177
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 39s 656ms/step - accuracy: 0.8921 - loss: 0.3001 - val_accuracy: 0.7987 - val_loss: 0.8788
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 39s 668ms/step - accuracy: 0.9038 - loss: 0.2958 - val_accuracy: 0.8051 - val_loss: 0.7371
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 40s 668ms/step - accuracy: 0.8844 - loss: 0.3171 - val_accuracy: 0.8437 - val_loss: 0.4151
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 40s 678ms/step - accuracy: 0.8869 - loss: 0.3161 - val_accuracy: 0.8116 - val_loss: 0.4820
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.9038 - loss: 0.2487 - val_accuracy: 0.

Test Accuracy: 87.50%
Model saved successfully!


In [4]:
import os
import pickle
import tensorflow as tf
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# 🔹 Paths Configuration
models_folder = r"C:\Users\VINIL\Desktop\soil_type_ML\flask_app\models"
os.makedirs(models_folder, exist_ok=True)

# Define file paths
cnn_model_path = os.path.join(models_folder, "soil_cnn_model.keras")  # .keras instead of .h5
class_indices_path = os.path.join(models_folder, "class_indices.pkl")

# 🔹 Save the Trained CNN Model in Keras Format
# Assuming you have a trained model already (named `model`)
model.save(cnn_model_path)
logging.info(f"CNN model saved successfully at: {cnn_model_path}")

# 🔹 Save class indices (from your train_generator)
# Assuming you have already loaded train_generator and it has class_indices
with open(class_indices_path, "wb") as ci_file:
    pickle.dump(train_generator.class_indices, ci_file)
logging.info(f"Class indices saved successfully at: {class_indices_path}")

print("CNN model and class indices saved successfully in:", models_folder)


CNN model and class indices saved successfully in: C:\Users\VINIL\Desktop\soil_type_ML\flask_app\models
